## Imports

In [18]:
import importlib
import io
import os
import shutil
import zipfile
from pathlib import Path
from typing import List

import requests

import java_parser

## Params

In [19]:
cloud_url = 'https://cloud.nemosoft.ru/s/fHLGN6ytDB3ziM4/download'
root_path = './docs2'

## Download from Nextcloud

In [20]:
response = requests.get(cloud_url, stream=True)
response.status_code

200

## Reading

In [21]:
zip_file = zipfile.ZipFile(io.BytesIO(response.content))

### File filtering

In [22]:
file_name_list: List[str] = [
    path for path in zip_file.namelist()
    if str(path).lower().endswith('.docx') or str(path).lower().endswith('.doc')
]

In [23]:
folder_name_list: List[str] = [
    path for path in zip_file.namelist()
    if str(path).lower().endswith('/')
]

## Parsing

In [24]:
importlib.reload(java_parser)
is_running = java_parser.check_if_parser_is_running()

Парсер уже запущен


In [25]:
if is_running:
    try:
        shutil.rmtree(root_path)
    except OSError:
        pass
    os.mkdir(root_path)

    for path in folder_name_list:
        Path(root_path + '/' + path).mkdir(parents=True, exist_ok=True)

    for name in file_name_list:
        try:
            list_of_docs = document = java_parser.send_document(
                zip_file.read(name),
                'DOC' if name.lower().endswith('doc') else 'DOCX'
            )
            for index, doc in enumerate(list_of_docs, start=1):
                final_text = ''
                for paragraph in doc['paragraphs']:
                    final_text += paragraph['paragraphHeader']['text'] + ' '
                    final_text += paragraph['paragraphBody']['text'] + ' '
                path = name + '_' + str(index) + '.txt'

                with open(root_path + '/' + path, 'w', encoding="utf-8") as f:
                    f.write(final_text)
        except Exception as e:
            print(name)
            print(e.with_traceback())

In [26]:
java_parser.kill_java_process()